In [1]:
from scipy import signal
import matplotlib.image as image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [2]:
# gradient linear

G_at_RB_conv = np.array([[0, 0, -1, 0, 0], 
                         [0, 0, 2, 0, 0], 
                         [-1, 2, 4, 2, -1], 
                         [0, 0, 2, 0, 0], 
                         [0, 0, -1, 0, 0]]) / 8
RB_at_G_conv1 = np.array([[0, 0, 1/2, 0, 0], 
                          [0, -1, 0, -1, 0], 
                          [-1, 4, 5, 4, -1], 
                          [0, -1, 0, -1, 0], 
                          [0, 0, 1/2, 0, 0]]) / 8
RB_at_G_conv2 = RB_at_G_conv1.T
RB_at_BR_conv = np.array([[0, 0, -3/2, 0, 0], 
                          [0, 2, 0, 2, 0], 
                          [-3/2, 0, 6, 0, -3/2], 
                          [0, 2, 0, 2, 0], 
                          [0, 0, -3/2, 0, 0]]) / 8

def Gradient_Demosaicing(img):
    length = img.shape[0]
    width = img.shape[1]
    padded_length = length + 4
    padded_width = width + 4
    
    # add padding to img for edge calculation
    padded_img = np.zeros((padded_length, padded_width))
    padded_img[2:padded_length-2, 2:padded_width-2] = img
    padded_img[0, 2:padded_width-2] = padded_img[2, 2:padded_width-2]
    padded_img[1, 2:padded_width-2] = padded_img[3, 2:padded_width-2]
    padded_img[padded_length-1, 2:padded_width-2] = padded_img[padded_length-3, 2:padded_width-2]
    padded_img[padded_length-2, 2:padded_width-2] = padded_img[padded_length-4, 2:padded_width-2]
    padded_img[:, 0] = padded_img[:, 2]
    padded_img[:, 1] = padded_img[:, 3]
    padded_img[:, padded_width-1] = padded_img[:, padded_width-3]
    padded_img[:, padded_width-2] = padded_img[:, padded_width-4]
    
    # calculate R, G, B:
    R = np.zeros((length, width))
    G = np.zeros((length, width))
    B = np.zeros((length, width))
    
    for i in range(length):
        for j in range(width):
            if i % 2 == j % 2:
                G[i][j] = img[i][j]
                if i % 2 == 0:
                    R[i][j] = signal.convolve2d(padded_img[i:i+5, j:j+5], RB_at_G_conv1, 'valid')[0][0]
                    B[i][j] = signal.convolve2d(padded_img[i:i+5, j:j+5], RB_at_G_conv2, 'valid')[0][0]
                else:
                    R[i][j] = signal.convolve2d(padded_img[i:i+5, j:j+5], RB_at_G_conv2, 'valid')[0][0]
                    B[i][j] = signal.convolve2d(padded_img[i:i+5, j:j+5], RB_at_G_conv1, 'valid')[0][0]
            else:
                G[i][j] = signal.convolve2d(padded_img[i:i+5, j:j+5], G_at_RB_conv, 'valid')[0][0]
                if i % 2 == 0:
                    R[i][j] = img[i][j]
                    B[i][j] = signal.convolve2d(padded_img[i:i+5, j:j+5], RB_at_BR_conv, 'valid')[0][0]
                else:
                    R[i][j] = signal.convolve2d(padded_img[i:i+5, j:j+5], RB_at_BR_conv, 'valid')[0][0]
                    B[i][j] = img[i][j]
                    
            j += 1
        i += 1
        
    # get a 3D output image
    # cv2 read and write image using BGR format
    img_d = np.zeros((length, width, 3))
    img_d[:, :, 0] = B / 255
    img_d[:, :, 1] = G / 255
    img_d[:, :, 2] = R / 255
    
    return img_d


In [ ]:
# process inputs

path = "/Users/qiaowenyang/Desktop/22ss/jssxx/labs/Codingproject/code/inputs"
def process(path):
    for file in os.listdir(path):
        filename = os.path.join(path, file)
        img = cv2.imread(filename)
        print(f"{file} before demosaicing:")
        cv2.imshow(f"{file}", img)
        cv2.waitKey(3000)
        
        img_cfa = img[:, :, 0] + img[:, :, 1] + img[:, :, 2]
        img_d = Gradient_Demosaicing(img_cfa)
        cv2.imwrite(f"gradient_outputs/demosaicing_{file}", img_d * 255)
        print(f"{file} after demosaicing:")
        cv2.imshow(f"demosaicing_{file}", img_d)
        cv2.waitKey(3000)
        cv2.destroyAllWindows()

process(path)

8.png before demosaicing:
8.png after demosaicing:
9.png before demosaicing:
9.png after demosaicing:
14.png before demosaicing:
14.png after demosaicing:
15.png before demosaicing:
15.png after demosaicing:
17.png before demosaicing:
17.png after demosaicing:
16.png before demosaicing:
16.png after demosaicing:
12.png before demosaicing:
12.png after demosaicing:
13.png before demosaicing:
13.png after demosaicing:
11.png before demosaicing:
11.png after demosaicing:
10.png before demosaicing:
10.png after demosaicing:
18.png before demosaicing:
18.png after demosaicing:
4.png before demosaicing:
4.png after demosaicing:
5.png before demosaicing:
5.png after demosaicing:
7.png before demosaicing:
7.png after demosaicing:
6.png before demosaicing:
